## Vime Self

In [1]:
import torch.optim as optim
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')

from seed_setting import *
from vime_model import VIME_SELF
from train_test import *
from MLP import MLP
from dataloader import *

from sklearn.metrics import f1_score

In [2]:
set_seed(2023)

seed is 2023...!


In [3]:
file_path = '/Users/kisehyun/Competition/데이콘 전화 해지 분류/train.csv'

In [4]:
lab_dl, unlab_dl, te_dl = get_data(file_path)

- lab_dl : labeled data
- unlab_dl : unlabeld data
- te_dl : test data

진행 과정
1. NN으로 lab_dl만 활용하여 학습 후 te_dl에 대해 성능 측정(macro f1 score)
2. vime self로 unlab_dl에 대해 pretext task 수행
3. 2에서 학습한 vime self model로 lab_dl feature extraction -> lab_dl 재생성
4. 1과 동일한 구조의 NN으로 te_dl에 대해 성능 측정

In [5]:
c1 = nn.BCELoss()
c2 = nn.L1Loss()

In [6]:
device = 'cpu'

In [7]:
num_epochs = 15
num_features = 12
batch_size = 128
threshold = 0.2
lr = 0.001
lambda_ = 1.2

***
### No Vime - Only Labeld Data

In [8]:
mlp_model = MLP(num_features)

In [9]:
optimizer = optim.Adam(mlp_model.parameters(), lr = lr)

In [10]:
### fit NN model using data loader using labeled data
no_vime_mlp_model = MLP_TRAIN(num_epochs, mlp_model, lab_dl, c1, threshold, optimizer, device)

1 Epoch Loss : 0.337119370318959 F1 score : 0.4870511065947042
2 Epoch Loss : 0.3340767331646942 F1 score : 0.4962162645597431
3 Epoch Loss : 0.3319673971499581 F1 score : 0.5082156950632761
4 Epoch Loss : 0.3279660540847894 F1 score : 0.5095996113478539
5 Epoch Loss : 0.3222991798912083 F1 score : 0.5148270311068385
6 Epoch Loss : 0.32012277835280983 F1 score : 0.5205187562789109
7 Epoch Loss : 0.31549625183795627 F1 score : 0.5217016069382883
8 Epoch Loss : 0.31691948743048903 F1 score : 0.5202026565027963
9 Epoch Loss : 0.3136299727130199 F1 score : 0.5255500279595254
10 Epoch Loss : 0.31065443467611975 F1 score : 0.527302047819013
11 Epoch Loss : 0.3113977341113022 F1 score : 0.5293336370908722
12 Epoch Loss : 0.3074371038294155 F1 score : 0.5307495184561403
13 Epoch Loss : 0.30397537126650753 F1 score : 0.5316680149007378
14 Epoch Loss : 0.30192102912539565 F1 score : 0.5335889584826162
15 Epoch Loss : 0.29922264899149453 F1 score : 0.5330571690676169


In [11]:
MLP_TEST_RESULT(no_vime_mlp_model, te_dl, threshold, device)

F1 Score is 0.4981043345679799


***
### Vime - Unlabeled Data

In [12]:
### vime self model
vime = VIME_SELF(num_features)

In [13]:
optimizer = optim.Adam(vime.parameters(), lr = lr)

In [14]:
### fit vime self model
vime_self_model = VIME_SELF_TRAIN(20, vime, unlab_dl, c1, c2, lambda_, threshold, optimizer, device)

1 Epoch Masking Loss : 57.76041666666666 & Feature Loss : 86.0673866853364
2 Epoch Masking Loss : 58.21180555555555 & Feature Loss : 40.00528522595466
3 Epoch Masking Loss : 56.42187499999998 & Feature Loss : 35.18440932060125
4 Epoch Masking Loss : 55.215277777777786 & Feature Loss : 29.23988653132294
5 Epoch Masking Loss : 55.61458333333333 & Feature Loss : 23.088028967715324
6 Epoch Masking Loss : 55.859375000000014 & Feature Loss : 19.798105389787466
7 Epoch Masking Loss : 55.86805555555555 & Feature Loss : 19.825440406217673
8 Epoch Masking Loss : 55.83680555555555 & Feature Loss : 19.155401947176784
9 Epoch Masking Loss : 56.0017361111111 & Feature Loss : 19.141561766208916
10 Epoch Masking Loss : 55.61458333333334 & Feature Loss : 18.76092885074296
11 Epoch Masking Loss : 55.45312500000002 & Feature Loss : 18.83892792073242
12 Epoch Masking Loss : 56.11111111111111 & Feature Loss : 18.21816252238025
13 Epoch Masking Loss : 55.61631944444445 & Feature Loss : 18.143028562858415
14

In [15]:
### extract feature using vime self model -> labeled data loader
vime_dl = VIME_SAMPLE(num_features, vime_self_model, lab_dl, batch_size, device)

In [16]:
vime_mlp_model = MLP(num_features)

In [17]:
optimizer = optim.Adam(vime_mlp_model.parameters(), lr = lr)

In [18]:
### fit NN model using data loader using vime self(labeled data)
vime_mlp_model = MLP_TRAIN(num_epochs, vime_mlp_model, vime_dl, c1, threshold, optimizer, device)

1 Epoch Loss : 0.380908796962892 F1 score : 0.45962417344156775
2 Epoch Loss : 0.3412714282438586 F1 score : 0.4933044746551814
3 Epoch Loss : 0.32859406939705976 F1 score : 0.49797347290297345
4 Epoch Loss : 0.3232200103947733 F1 score : 0.5024395236584362
5 Epoch Loss : 0.3148803114792421 F1 score : 0.5049430646089341
6 Epoch Loss : 0.3157720383529824 F1 score : 0.5076000345641145
7 Epoch Loss : 0.3137837983471995 F1 score : 0.5073347050102359
8 Epoch Loss : 0.31363661386219616 F1 score : 0.5092706326668062
9 Epoch Loss : 0.31435793159560077 F1 score : 0.5109737940390516
10 Epoch Loss : 0.31692516084802747 F1 score : 0.5117961748638337
11 Epoch Loss : 0.3157772640267277 F1 score : 0.514887619607894
12 Epoch Loss : 0.31591077452944344 F1 score : 0.5154226808992205
13 Epoch Loss : 0.3158651264390939 F1 score : 0.5179224127792175
14 Epoch Loss : 0.3153740469549539 F1 score : 0.5185348933102512
15 Epoch Loss : 0.31286726092089684 F1 score : 0.5174150596817955


In [19]:
MLP_TEST_RESULT(vime_mlp_model, te_dl, threshold, device)

F1 Score is 0.5464769835487452
